In [127]:
from selenium import webdriver
import datetime, re, requests, io, time, random, string
from bs4 import BeautifulSoup
from credentials import email, password
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

In [128]:
urls = {
      'sign_in': 'https://wallmine.com/users/sign-in',
      'homepage': 'https://wallmine.com/',
      'view_stock': lambda exchange, symbol: f'https://wallmine.com/{exchange}/{symbol}'
     }

#urls['view_stock']('nasdaq','tsla')

In [129]:
email

'erikdotg@gmail.com'

In [130]:
driver = webdriver.Chrome(r"C:\Users\Erik\Downloads\chromedriver.exe")

In [131]:
def login():
    '''Method used to login into Wallmine account'''
    driver.get(urls['sign_in'])
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]/a').click()
    time.sleep(0.5)
    driver.find_element_by_xpath('//*[@id="user_email"]').send_keys(email)
    driver.find_element_by_xpath('//*[@id="user_password"]').send_keys(password)
    time.sleep(0.1)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button').click()
    time.sleep(3)
    if driver.find_element_by_xpath('/html/body/main/section/div[2]/div/div[1]/h1/span'):
        print('Login was successful')

In [132]:
login()

Login was successful


In [76]:
def retrieve_data():
    '''Method use to retrieve stock data from Wallmine'''
    # We need to be on the site after login in order for this method to work
#     driver.find_element_by_xpath('/html/body/main/section/div[3]/div[2]/div[1]/div[3]/div/div/a').click()
    driver.get('https://wallmine.com/screener?e%5B%5D=NYSE&e%5B%5D=NASDAQ&e%5B%5D=NYSEMKT&hm=performance_today_with_ah&r=m')
    time.sleep(5)
    text = driver.find_element_by_xpath('/html/body/main/section/div[4]/div/div/div[1]/div/div[1]/h1').text
    if text == 'Free Stock Screener':
        print('On the right page')
#     driver.find_element_by_xpath('/html/body/main/section/div[5]/form/div/div').click()
#     time.sleep(1)
#     driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[1]/div/ul/li[1]/a').click() # click on overview tab
    driver.get('https://wallmine.com/screener?e%5B%5D=NASDAQ&e%5B%5D=NYSE&e%5B%5D=NYSEMKT&r=o&o=m&d=d&hm=performance_today_with_ah&fo=e%5B%5D')
    time.sleep(2)
    table_head = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[2]/table/thead').text
    table_body = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[2]/table/tbody').text
    raw_data = [table_head, table_body]
    return raw_data
    

In [77]:
retrieve_data()

On the right page


['▴ ▴\nSymbol\n▴ ▴\nCompany\n▴ ▴\nExchange\n▴ ▴\nIndustry\n▴ ▴\nMarket cap\n▴ ▴\nEBITDA\n▴ ▴\nP/E\n▴ ▴\nEV/EBITDA\n▴ ▴\nDebt/Equity\n▴ ▴\nAverage volume\n▴ ▴\nInstitutional ownership\n▴ ▴\nEarnings date\n▴ ▴\nPrice\n▴ ▴\nPerformance today',
 'AAPL Apple Inc NASDAQ Consumer Electronics $1.97T $85.159B 32.37 24.07 3.96 113.145M 59.66% 27 Jan 2021 $121.39 +0.15%\nMSFT Microsoft Corporation NASDAQ Internet Services & Infrastructure $1.76T $60.155B 34.88 27.69 1.55 30.361M 71.84% 26 Jan 2021 $235.11 -0.58%\nAMZN Amazon.com Inc. NASDAQ Specialty Retail $1.53T $48.091B 71.58 31.10 2.44 3.834M 58.66% 2 Feb 2021 $3,075.73 +0.78%\nGOOGL Alphabet Inc NASDAQ Internet Services & Infrastructure $1.38T $55.322B 34.23 22.72 0.44 N/A 80.25% 2 Feb 2021 $2,045.79 +1.04%\nGOOG Alphabet Inc NASDAQ Internet Services & Infrastructure $1.38T N/A 41.05 N/A 0.44 1.613M 68.10% 2 Feb 2021 $2,055.95 +1.00%\nFB Facebook Inc NASDAQ Internet Services & Infrastructure $805.95B $39.533B 28.55 19.38 0.24 19.371M 79.77% 

In [78]:
got_data = retrieve_data()
got_data

On the right page


['▴ ▴\nSymbol\n▴ ▴\nCompany\n▴ ▴\nExchange\n▴ ▴\nIndustry\n▴ ▴\nMarket cap\n▴ ▴\nEBITDA\n▴ ▴\nP/E\n▴ ▴\nEV/EBITDA\n▴ ▴\nDebt/Equity\n▴ ▴\nAverage volume\n▴ ▴\nInstitutional ownership\n▴ ▴\nEarnings date\n▴ ▴\nPrice\n▴ ▴\nPerformance today',
 'AAPL Apple Inc NASDAQ Consumer Electronics $1.97T $85.159B 32.37 24.07 3.96 113.145M 59.66% 27 Jan 2021 $121.39 +0.15%\nMSFT Microsoft Corporation NASDAQ Internet Services & Infrastructure $1.76T $60.155B 34.88 27.69 1.55 30.361M 71.84% 26 Jan 2021 $235.11 -0.58%\nAMZN Amazon.com Inc. NASDAQ Specialty Retail $1.53T $48.091B 71.58 31.10 2.44 3.834M 58.66% 2 Feb 2021 $3,075.73 +0.78%\nGOOGL Alphabet Inc NASDAQ Internet Services & Infrastructure $1.38T $55.322B 34.23 22.72 0.44 N/A 80.25% 2 Feb 2021 $2,045.79 +1.04%\nGOOG Alphabet Inc NASDAQ Internet Services & Infrastructure $1.38T N/A 41.05 N/A 0.44 1.613M 68.10% 2 Feb 2021 $2,055.95 +1.00%\nFB Facebook Inc NASDAQ Internet Services & Infrastructure $805.95B $39.533B 28.55 19.38 0.24 19.371M 79.77% 

In [79]:
got_data[0]

'▴ ▴\nSymbol\n▴ ▴\nCompany\n▴ ▴\nExchange\n▴ ▴\nIndustry\n▴ ▴\nMarket cap\n▴ ▴\nEBITDA\n▴ ▴\nP/E\n▴ ▴\nEV/EBITDA\n▴ ▴\nDebt/Equity\n▴ ▴\nAverage volume\n▴ ▴\nInstitutional ownership\n▴ ▴\nEarnings date\n▴ ▴\nPrice\n▴ ▴\nPerformance today'

In [80]:
clean_data_head = got_data[0].split('\n')
clean_data_head

['▴ ▴',
 'Symbol',
 '▴ ▴',
 'Company',
 '▴ ▴',
 'Exchange',
 '▴ ▴',
 'Industry',
 '▴ ▴',
 'Market cap',
 '▴ ▴',
 'EBITDA',
 '▴ ▴',
 'P/E',
 '▴ ▴',
 'EV/EBITDA',
 '▴ ▴',
 'Debt/Equity',
 '▴ ▴',
 'Average volume',
 '▴ ▴',
 'Institutional ownership',
 '▴ ▴',
 'Earnings date',
 '▴ ▴',
 'Price',
 '▴ ▴',
 'Performance today']

In [81]:
def remove_characters(x):
    if x == '▴ ▴':
        return False
    else:
        return True

In [82]:
categories = list(filter(remove_characters, clean_data_head))
categories

['Symbol',
 'Company',
 'Exchange',
 'Industry',
 'Market cap',
 'EBITDA',
 'P/E',
 'EV/EBITDA',
 'Debt/Equity',
 'Average volume',
 'Institutional ownership',
 'Earnings date',
 'Price',
 'Performance today']

In [83]:
got_data[1]

'AAPL Apple Inc NASDAQ Consumer Electronics $1.97T $85.159B 32.37 24.07 3.96 113.145M 59.66% 27 Jan 2021 $121.39 +0.15%\nMSFT Microsoft Corporation NASDAQ Internet Services & Infrastructure $1.76T $60.155B 34.88 27.69 1.55 30.361M 71.84% 26 Jan 2021 $235.11 -0.58%\nAMZN Amazon.com Inc. NASDAQ Specialty Retail $1.53T $48.091B 71.58 31.10 2.44 3.834M 58.66% 2 Feb 2021 $3,075.73 +0.78%\nGOOGL Alphabet Inc NASDAQ Internet Services & Infrastructure $1.38T $55.322B 34.23 22.72 0.44 N/A 80.25% 2 Feb 2021 $2,045.79 +1.04%\nGOOG Alphabet Inc NASDAQ Internet Services & Infrastructure $1.38T N/A 41.05 N/A 0.44 1.613M 68.10% 2 Feb 2021 $2,055.95 +1.00%\nFB Facebook Inc NASDAQ Internet Services & Infrastructure $805.95B $39.533B 28.55 19.38 0.24 19.371M 79.77% 27 Jan 2021 $290.82 +2.76%\nBABA Alibaba Group Holding Ltd NYSE Specialty Retail $603.32B ¥17.718B 25.26 220.15 0.57 26.259M 43.86% 2 Feb 2021 $231.87 +2.03%\nTSLA Tesla Inc NASDAQ Automobile Manufacturers $593.87B $4.316B 836.22 143.87 1.28 

In [84]:
clean_data_body = got_data[1].split('\n')
clean_data_body[0]

'AAPL Apple Inc NASDAQ Consumer Electronics $1.97T $85.159B 32.37 24.07 3.96 113.145M 59.66% 27 Jan 2021 $121.39 +0.15%'

In [85]:
split_company = clean_data_body[0].split('$')
split_company

['AAPL Apple Inc NASDAQ Consumer Electronics ',
 '1.97T ',
 '85.159B 32.37 24.07 3.96 113.145M 59.66% 27 Jan 2021 ',
 '121.39 +0.15%']

In [86]:
# we want:
# symbol,
# company name
# exchange,
# price
# and store them all inside an object

In [87]:
split_row_one = split_company[0].split(' ')
split_row_two = split_company[1].split(' ')
split_row_three = split_company[2].split(' ')
split_row_four = split_company[3].split(' ')
stop = ['NASDAQ', 'NYSE', 'NYSEMKT']

symbol = split_row_one[0]
price = split_row_four[0]
for n in stop:
    if n in split_company[0]:
        exchange = n
a = split_row_one.index(exchange)
company_name_arr = split_row_one[1:a]
company_name = ''.join(company_name_arr)

print(company_name_arr)
print(exchange)
print(symbol)
print(price)

['Apple', 'Inc']
NASDAQ
AAPL
121.39


In [88]:
def parse_company(company):
    result={}
    split_row_one = split_company[0].split(' ')
    split_row_two = split_company[1].split(' ')
    split_row_three = split_company[2].split(' ')
    split_row_four = split_company[3].split(' ')
    stop = ['NASDAQ', 'NYSE', 'NYSEMKT']

    symbol = split_row_one[0]
    price = split_row_four[0]
    for n in stop:
        if n in split_company[0]:
            exchange = n
    a = split_row_one.index(exchange)
    company_name_arr = split_row_one[1:a]
    # company_name = ' '.join(str(e) for e in company_name_arr)
    company_name = ' '.join(company_name_arr)
    
    result['symbol'] = symbol
    result['company_name'] = company_name
    result['exchange'] = exchange
    result['price'] = price
    result['date'] = datetime.datetime.now()
           
    return result


In [89]:
all_company_data = []
for stock in clean_data_body:
    if 'N/A' in stock:
        continue
    elif '¥' in stock:
        continue
    elif '€' in stock:
        continue
    split_company = stock.split('$')
    
    x = parse_company(split_company)
    all_company_data.append(x)

In [90]:
final_list = [categories, all_company_data]
final_list

[['Symbol',
  'Company',
  'Exchange',
  'Industry',
  'Market cap',
  'EBITDA',
  'P/E',
  'EV/EBITDA',
  'Debt/Equity',
  'Average volume',
  'Institutional ownership',
  'Earnings date',
  'Price',
  'Performance today'],
 [{'symbol': 'AAPL',
   'company_name': 'Apple Inc',
   'exchange': 'NASDAQ',
   'price': '121.39',
   'date': datetime.datetime(2021, 3, 29, 15, 27, 52, 709415)},
  {'symbol': 'MSFT',
   'company_name': 'Microsoft Corporation',
   'exchange': 'NASDAQ',
   'price': '235.11',
   'date': datetime.datetime(2021, 3, 29, 15, 27, 52, 709415)},
  {'symbol': 'AMZN',
   'company_name': 'Amazon.com Inc.',
   'exchange': 'NASDAQ',
   'price': '3,075.73',
   'date': datetime.datetime(2021, 3, 29, 15, 27, 52, 709415)},
  {'symbol': 'FB',
   'company_name': 'Facebook Inc',
   'exchange': 'NASDAQ',
   'price': '290.82',
   'date': datetime.datetime(2021, 3, 29, 15, 27, 52, 709415)},
  {'symbol': 'BRK-B',
   'company_name': 'Berkshire Hathaway Inc.',
   'exchange': 'NYSE',
   'pri

In [91]:
db = client.stock_database

#def store_data(parsed_data):

categories = db.categories
stocks = db.stocks

In [92]:
# insert each category
def store_data(final_list):
    categories = db.categories
    stocks = db.stocks
    c = final_list[0]
    for item in c:
        category = {
            'category': item
        }
        categories.insert_one(category)
    stocks.insert_many(final_list[1])
    return 'Items added to DB'

In [133]:
def view_stock(exchange, symbol):
    website = urls['view_stock'](exchange, symbol)
    driver.get(website)
    time.sleep(3)
    if '404' in driver.page_source and "Sorry, we couldn't find the page you requested" in driver.page_source:
        return {
            'exchange': exchange,
            'symbol': symbol,
            'message': 'Please double check the exchange, and symbol. Try again'
        }
    symbol = driver.find_element_by_xpath('/html/body/main/section/div[2]/div/div[1]/h1/div[1]').text
    exchange = driver.find_element_by_xpath('/html/body/main/section/div[2]/div/div[1]/div/nav/ol/li[1]/a').text
    company_name = driver.find_element_by_xpath('/html/body/main/section/div[2]/div/div[1]/h1/div[1]').text
    price = driver.find_element_by_xpath('/html/body/main/section/div[3]/div/div[1]/div/div/div[2]/div/div[1]/span[1]').text
    result = {
        'symbol': symbol,
        'exchange': exchange,
        'company_name': company_name,
        'price': price
    }

    return result

In [134]:
view_stock('nasdaq','tsla')

{'symbol': 'TSLA',
 'exchange': 'NASDAQ',
 'company_name': 'TSLA',
 'price': '$611.32'}

In [167]:
def compare_price(symbol):
    symbol = symbol.upper()
    s = stocks.find_one({'symbol': symbol})
    website = urls['view_stock'](s['exchange'], symbol)
    driver.get(website)
    time.sleep(3)
    current_price = float(driver.find_element_by_xpath('/html/body/main/section/div[3]/div/div[1]/div/div/div[2]/div/div[1]/span[1]').text[1:])
    previous_price = float(s.get('price'))
    print(current_price, previous_price)
    return current_price - previous_price

In [168]:
compare_price('aapl')

121.39 121.39


0.0